<a href="https://colab.research.google.com/github/ikm4rkov/OCR_examples/blob/main/OCR_comparing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Существует множество (десятки [1]) инструментов для распознавания письменной речи (optical character recognition; OCR), но для данного сравнения будут использованы только бесплатные инструменты с поддержкой русского языка. В сравнении приняли участие Tesseract [2], Easy-OCR [3] и surya [4].

Алгоритм Tesseract написан на C++ и может быть использован как исполнимый файл.

In [1]:
!sudo apt install tesseract-ocr
!sudo apt install tesseract-ocr-rus

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (3,438 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debc

Но также он может быть включен как модуль в Python.

In [2]:
!pip install pytesseract

Все параметры конфигурационного файла модуля входят имеются в конфигурационном файле исполнимого файла [5], но не все могут быть изменены. Поэтому провариированы только параметры для модуля Python.

Основные параметры модуля Pytesseract [6]:

1.   --lang языки распознавания; доступны около 100 языков, в том числе русский, можно совмещать несколько, например '--lang rus+eng'; требует в /usr/share/tesseract-ocr/4.00/tessdata/ модель для каждого языка; по-умолчанию eng
2.   --psm (Page Segmentation Mode) режим структуры текста, которую Tesseract ,будет ожидать

*   0 ориентация и детекция текста (без распознавания)
*   1 автоматическая сегментация страницы с определением ориентации текста
*   2 автоматическая сегментация страницы без определения ориентации текста (утверждается, что не работает)
*   3 полностью автоматическая сегментация страницы (ожидается одна или несколько колонок текста на фото) без определения ориентации текста; это значение по-умолчанию
*   4 текст в виде одного столбца различной ширины
*   5 единственный вертикально выровненный блок текста
*   6 единственный блок текста
*   7 единственная строка
*   8 единственное слово
*   9 единственное слово в кругу
*   10 единственный символ
*   11 разрозненный текст в случайном порядке
*   12 разрозненный текст в случайном порядке с определением ориентации
*   13 единственная строка в "сыром виде"

3.   --oem (OCR engine Mode) выбор движка для определения, доступны 4 значения:

*   0 старый движок Tesseract
*   1 нейронные сети LSTM (косвенно по-умолчанию)
*   2 старый движок Tesseract и нейронные сети LSTM
*   3 выбирает то, что имеется (по-умолчанию)

Остальные не влияют на алгоритм, затрагивая только производительность и формат выходных данных.

Так как по-умолчанию единственный доступный движок - нейронные сети LSTM, опция --oem принимает единственное значение.

Принимая во внимание, что документы, скорее всего, будут на русском и английском языках опции --lang было присвоено значение rus+eng.

Для параметра структуры текста были провариированы все значения, кроме 0, для которого невозможно оценить точность разпознавания и 2, так как при этом значении pytesseract не заработал. Для того, чтобы работала ориентация текста необходимо также загрузить osd.traineddata.

В качестве тестов были взяты 3 изображения: первое - первый разворот паспорта, второе - тот же разворот, но повернутый на 45 градусов и третий - примерно половина стихотворения Лермонтова "Бородино". В качестве метрик точности были выбраны редакционное расстояние по Левенштейну [7] предсказанного текста с предварительно указанным и коэффициент сходства Серенсена (отношение двух мощностей пересечения множеств к сумме мощностей) [8], а в качестве метрики скорости - время работы.




In [3]:
!pip install Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 40.6 MB/s eta 0:00:00


In [5]:
import time
import cv2
import pytesseract
import Levenshtein as lev
import collections
import re
from itertools import product

image_paths = ["passport1.png", "passport2.jpg", "borodino.png"]
ground_truth = {
    "passport1.png": "ПАСПОРТ ГРАЖДАНИНА РОССИЙСКОЙ ФЕДЕРАЦИИ",
    "passport2.jpg": "ПАСПОРТ ГРАЖДАНИНА РОССИЙСКОЙ ФЕДЕРАЦИИ",
    "borodino.png": "Бородино стихи М. Ю. Лермонтова -Скажи-ка, дядя, ведь не даром Москва, спаленная пожаром, Франузу отдана? Ведь были ж схватки боевые, Да, говорят, еще какие! Не даром помнит вся Россия Про день Бородина! -Да, были люди в наше время, Не то что нынешнее племя: Богатыри - не вы! Плохая им досталась доля: Немногие вернулись с поля... Не будь на то господня воля, Не отдали б Москвы! Мы долго молча отступали, Досадно было, боя ждали, Ворчали старики: Что ж мы? на зимние квартиры? Не смеют, что ли, командиры Чужие изорвать мундиры О русские штыки? И вот нашли большое поле: Есть разгуляться где на воле! Построили редут. У наших ушки на макушке! Чуть утро осветило пушки И леса синие верхушки - Французы тут как тут. Забил снаряд я в пушку туго И думал: угощу я друга! Постой-ка, брат мусью! Что тут хитрить, пожалуй к бою; Уж мы пойдем ломить стеною, Уж постоим мы головою За родину свою!"
}

def clean_text(text):
    return re.sub(r'[^\w\s]', '', text.lower())

def get_word_counter(text):
    words = text.split()
    return collections.Counter(words)

def calculate_word_similarity(text1, text2):
    clean_text1 = clean_text(text1)
    clean_text2 = clean_text(text2)

    counter1 = get_word_counter(clean_text1)
    counter2 = get_word_counter(clean_text2)

    intersection = sum((counter1 & counter2).values())

    similarity = 2 * intersection / (sum(counter1.values()) + sum(counter2.values()))
    return similarity

def perform_ocr_with_tesseract(image_path, config=""):
    image = cv2.imread(image_path)
    text = pytesseract.image_to_string(image, lang='rus+eng', config=config)
    text = text.replace("\n", " ").replace("\r", " ").strip()
    return text

def calculate_levenshtein_distance(reference, hypothesis):
    return lev.distance(reference, hypothesis)

config_params = {
    "psm": [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
    "oem": [3],
}

for image_path in image_paths:
    print(f"Обработка изображения: {image_path}")
    ground_truth_text = ground_truth[image_path]

    for config_values in product(*config_params.values()):
        config_string = " ".join(
            [f"--{key} {value}" for key, value in zip(config_params.keys(), config_values)]
        )

        start_time = time.time()
        tesseract_text = perform_ocr_with_tesseract(image_path, config=config_string)
        tesseract_time = time.time() - start_time

        levenshtein_distance = calculate_levenshtein_distance(ground_truth_text, tesseract_text)

        word_similarity = calculate_word_similarity(ground_truth_text, tesseract_text)

        print(f"Конфигурация: {config_string}")
        print(f"Tesseract OCR - Время: {tesseract_time:.2f} сек")
        print("Распознанный текст (Tesseract):")
        print(tesseract_text)
        print(f"Tesseract - Абсолютное расстояние Левенштейна: {levenshtein_distance}")
        print(f"Tesseract - Сходство по словам (метрика Жаккара): {word_similarity:.4f}")
        print("-" * 50)


Обработка изображения: passport1.png
Конфигурация: --psm 1 --oem 3
Tesseract OCR - Время: 0.86 сек
Распознанный текст (Tesseract):
ГРАЖДАНИНА \CCUMCKOV ФЕДЕРАЦИИ
Tesseract - Абсолютное расстояние Левенштейна: 18
Tesseract - Сходство по словам (метрика Жаккара): 0.5714
--------------------------------------------------
Конфигурация: --psm 3 --oem 3
Tesseract OCR - Время: 0.43 сек
Распознанный текст (Tesseract):
ГРАЖДАНИНА \CCUMCKOV ФЕДЕРАЦИИ
Tesseract - Абсолютное расстояние Левенштейна: 18
Tesseract - Сходство по словам (метрика Жаккара): 0.5714
--------------------------------------------------
Конфигурация: --psm 4 --oem 3
Tesseract OCR - Время: 0.46 сек
Распознанный текст (Tesseract):
ГРАЖДАНИНА \CCUMCKOV ФЕДЕРАЦИИ
Tesseract - Абсолютное расстояние Левенштейна: 18
Tesseract - Сходство по словам (метрика Жаккара): 0.5714
--------------------------------------------------
Конфигурация: --psm 5 --oem 3
Tesseract OCR - Время: 2.04 сек
Распознанный текст (Tesseract):
S, а А | 235 ae ЕЕ №

Структура текста во всех трех случаях может быть описана как одна или несколько колонок, за исключением passport2.jpg, где текст повернут на 45 градусов, поэтому ожидаемо --psm 3 в первом и третьем текстах показали лучшую точность (наименьшее редакционное расстояние) и примерно лучшую скорость (--psm 1 способен распознать ту же структуру, как у --psm 3, но обычно за бОльшее время). Для текста "Бородино" около 92% всех слов распознались верно. Тем не менее, даже при определении ориентации текста, модуль Tesseract не смог правильно распознать повернутый на 45 градусов текст, одну букву смогли распознать только опции с рассеяным текстом.

In [6]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 20.9 MB/s eta 0:00:00


In [7]:
import time
import easyocr
import Levenshtein as lev
import re
import collections

image_paths = ["passport1.png", "passport2.jpg", "borodino.png"]
ground_truth = {
    "passport1.png": "ПАСПОРТ ГРАЖДАНИНА РОССИЙСКОЙ ФЕДЕРАЦИИ",
    "passport2.jpg": "ПАСПОРТ ГРАЖДАНИНА РОССИЙСКОЙ ФЕДЕРАЦИИ",
    "borodino.png": "Бородино стихи М. Ю. Лермонтова -Скажи-ка, дядя, ведь не даром Москва, спаленная пожаром, Франузу отдана? Ведь были ж схватки боевые, Да, говорят, еще какие! Не даром помнит вся Россия Про день Бородина! -Да, были люди в наше время, Не то что нынешнее племя: Богатыри - не вы! Плохая им досталась доля: Немногие вернулись с поля... Не будь на то господня воля, Не отдали б Москвы! Мы долго молча отступали, Досадно было, боя ждали, Ворчали старики: Что ж мы? на зимние квартиры? Не смеют, что ли, командиры Чужие изорвать мундиры О русские штыки? И вот нашли большое поле: Есть разгуляться где на воле! Построили редут. У наших ушки на макушке! Чуть утро осветило пушки И леса синие верхушки - Французы тут как тут. Забил снаряд я в пушку туго И думал: угощу я друга! Постой-ка, брат мусью! Что тут хитрить, пожалуй к бою; Уж мы пойдем ломить стеною, Уж постоим мы головою За родину свою!"
}

def clean_text(text):
    return re.sub(r'[^\w\s]', '', text.lower())

def get_word_counter(text):
    words = text.split()
    return collections.Counter(words)

def calculate_jaccard_similarity(text1, text2):
    clean_text1 = clean_text(text1)
    clean_text2 = clean_text(text2)
    counter1 = get_word_counter(clean_text1)
    counter2 = get_word_counter(clean_text2)
    intersection = sum((counter1 & counter2).values())
    similarity = 2 * intersection / (sum(counter1.values()) + sum(counter2.values()))
    return similarity

def perform_ocr_with_easyocr(image_path):
    reader = easyocr.Reader(['ru'])
    result = reader.readtext(image_path)
    text = " ".join([res[1] for res in result])
    text = text.replace("\n", " ").replace("\r", " ").strip()
    return text

def calculate_levenshtein_distance(reference, hypothesis):
    return lev.distance(reference, hypothesis)

for image_path in image_paths:
    print(f"Обработка изображения: {image_path}")
    ground_truth_text = ground_truth[image_path]

    start_time = time.time()
    easyocr_text = perform_ocr_with_easyocr(image_path)
    easyocr_time = time.time() - start_time

    levenshtein_distance = calculate_levenshtein_distance(ground_truth_text, easyocr_text)
    jaccard_similarity = calculate_jaccard_similarity(ground_truth_text, easyocr_text)

    print(f"EasyOCR - Время: {easyocr_time:.2f} сек")
    print("Распознанный текст (EasyOCR):")
    print(easyocr_text)
    print(f"EasyOCR - Абсолютное расстояние Левенштейна: {levenshtein_distance}")
    print(f"EasyOCR - Сходство по словам (Жаккар): {jaccard_similarity:.4f}")
    print("-" * 50)

Обработка изображения: passport1.png
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.1% Complete

EasyOCR - Время: 19.08 сек
Распознанный текст (EasyOCR):
[ЯШФТ ГРА Ж АНИН; Российской ФЕДЕРАЦИИ
EasyOCR - Абсолютное расстояние Левенштейна: 18
EasyOCR - Сходство по словам (Жаккар): 0.4000
--------------------------------------------------
Обработка изображения: passport2.jpg


EasyOCR - Время: 26.08 сек
Распознанный текст (EasyOCR):
ЛАСПОРТ ФЕАЕРАЦИИ ГРАЖААНИНА Российской
EasyOCR - Абсолютное расстояние Левенштейна: 28
EasyOCR - Сходство по словам (Жаккар): 0.2500
--------------------------------------------------
Обработка изображения: borodino.png
EasyOCR - Время: 20.54 сек
Распознанный текст (EasyOCR):
Бородино Что ж мы? на зимние квартиры? стихи М. Ю. Лермонтова Не смеют что лИ, командиры Скажи-ка, дядя, ведь не даром Чужие изорвать мундиры Москва, спаленная пожаром, О русские штыки?  Французу отдана? И вот нашли большое поле: Ведь были ж схватки боевые, Есть разгуляться где на воле ! Да, говорят, еше какие ! Построили редут. Недаром помнит вся Россия У наших ушки на макушке ! Про день Бородина ! утро осветило пушки Да, были люди в наше время И леса синие верхушки Не то, что нынешнее племя: Французы тут как тут. Богатыри не вы ! Забил заряд я в пушку туго Плохая им досталась доля: И думал: угощу я друга ! Немногие вернулись ПОЛЯ Постой-ка, брат мусью ! Н

EasyOCR позиционирует себя как небольшая простая в использовании библиотека, поэтому единственным кастомизируемым параметром в ней является выбор модели. Время работы инструмента больше, чем у Tesseract на пол-, один порядка. EasyOCR умеет распознавать текст, но она плохо справляется с группировкой текста в блоки. Помимо самого текста сохраняет координаты 4 вершин прямоугольника с распознанным текстом в res[0]. Но даже так рамки не всегда точны, попытка построить центр прямоугольника (среднее арифметическое координат по обеим осям) и упорядочить текст слева-направо сверху-вниз (или сверху-вниз слева-направо для данных текстов в виде нескольких колонок) не привела к получению правильного порядка и уменьшению редакционного растояния. Так,  на тексте "Бородино" расстояние Левенштейна примерно в 2 раза больше, чем у Tesseract в лучшей конфигурации, при этом метрика доли сходства распознанныз слов у EasyOCR даже больше, чем у Tesseract. Из очевидных плюсов - EasyOCR способна распознавать развернутый текст (и ограничено рукописный).

In [8]:
!pip install surya-ocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.0 MB/s eta 0:00:00


Surya OCR может быть интегрирована в Python, но модуль использует свои типы, и требуется время, чтобы разобраться как правильно с ними работать. Поэтому в этом сравнении была использована консольная версия. Она запускается как surya_ocr "file" и требуется указание языка --langs, которое можно также задавать в отдельном файле. На выход подается .JSON файл result.json по-умолчанию по адресу results/surya/"file"/results.json, в котором также хранятся координаты четырех углов рамки распознанного фрагмента текста. Помимо непосредственно surya-ocr имеются другие инструменты, например, анализ разметки surya_layout.

In [9]:
!for file in *.{jpeg,jpg,png,pdf}; do [ -f "$file" ] && { start=$(date +%s); surya_ocr --langs ru "$file"; end=$(date +%s); echo "Время обработки $file: $((end - start)) секунд"; }; done


2024-11-08 13:25:57.621366: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-08 13:25:57.990082: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-08 13:25:58.102010: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-08 13:25:58.733503: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-08 13:26:00.649732: W tensorflow/compiler/tf2

Программа работает дольше всего из сравниваемых, также для нее нужна обработка .json файла, которая будет рассчитывать метрики, реализация ниже (surya_json_parser_borodino.py отличается только тем, что в ground_truth_text записан текст стихотворения).

In [15]:
!cat surya_json_parser.py

import os
import json
import Levenshtein as lev
import sys
import re
import collections

def clean_text(text):
      return re.sub(r'[^\w\s]', '', text.lower())

def get_word_set(text):
    words = clean_text(text).split()
    return set(words)

def calculate_jaccard_similarity(text1, text2):
    set1 = get_word_set(text1)
    set2 = get_word_set(text2)
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union != 0 else 0.0

def process_file(file_path, ground_truth_text):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
        page_data = list(data.values())[0][0]['text_lines'] 
        
        text_fragments = [item['text'] for item in page_data]
    
    combined_text = " ".join(text_fragments).strip()
    lev_distance = lev.distance(ground_truth_text, combined_text)
    jaccard_similarity = calculate_jaccard_similarity(ground_truth_text, combined_text)
    return combined_text, lev_distance, jacc

In [11]:
!python surya_json_parser.py results/surya/passport1/results.json

Объединённый текст:
ПАСПОРТ ГРАЖДАНИНА РОССИЙСКОЙ ФЕДЕРАЦИИ
Расстояние Левенштейна: 0
Сходство по Жаккару: 1.0000


In [12]:
!python surya_json_parser.py results/surya/passport2/results.json

Объединённый текст:
метрования соста и привается собедния подать ПАСПОРТ РОССИЙСКОЙ ФЕДЕРАЦИИ
Расстояние Левенштейна: 50
Сходство по Жаккару: 0.3000


In [14]:
!python surya_json_parser_borodino.py results/surya/borodino/results.json

Объединённый текст:
Бородинс «Что ж мы? на зимние квартиры стихи М. Ю. Лермонтова Не смеют, что ли, командиры Чужие изорвать мундиры Скажи-ка, дядя, ведь не даром О русские штыки?» Москва, спаленная пожаром, И вот нашли большое поле: Французу отдана? Есть разгуляться где на воле! Ведь были ж схватки боевые, Построили редут. Да, говорят, еще какие! У наших ушки на макушке! Недаром помнит вся Россия Чуть утро осветило пушки Про день Бородина! И леса синие верхушки - - Да, были люди в наше время, Французы тут как тут. Не то, что нынешнее племя: Забил заряд я в пушку туго Богатыри - не вы! И думал: угощу я друга! Плохая им досталась доля: Постой-ка, брат мусью! Немногие вернулись с поля ... Что тут хитрить, пожалуй к бою; Не будь на то господня воля, Уж мы пойдем ломить стеною, Не отдали б Москвы! Уж постоим мы головою Мы долго молча отступали, За родину свою! Досадно было, боя ждали, Ворчали старики:
Расстояние Левенштейна: 715
Сходство по Жаккару: 0.9453


В результате: производительность лучше всего у Tesseract, примерно на пол-порядка дольше работает EasyOCR, и еще на порядок дольше работает surya. Tesseract немного проигрывает по обеим метрикам на изображениях паспорта, но примерно в 2 раза выигрывает по редакционному расстоянию на стихе "Бородино". EasyOCR и surya лучше могут распознавать повернутый текст, но не могут без дополнительных программ объединить найденные фрагменты текста в правильном порядке (обе программы сохраняют координаты рамок). Поэтому при достаточно высокой степени распознования слов (около 95% слов были распознаны правильно) EasyOCR и surya значительно проигрывают по расстоянию Левенштейна.

Tesseract в виду своей произоводительности и относительно неплохой точности на текстах, чья структура оптимальна для программы, может быть использован с наименьшими затрами. Но  он требует правильной ориентации текста и желательно или приведения к нескольким столбцам, или фрагментации на отдельные блоки/строки/символы. В расширенном конфиге для исполнимого файла содержатся много параметров: ограничивающие шрифт и словарь, задающие весовые коэффициенты модели и устанавливающие режимы. Также Tesseract позволяет дообучивать и добавлять собственные модели. Это делает инструмент достаточно гибким под конкретную задачу, но необходим вспомогательный код, и все еще может потребоваться модерация.

EasyOCR обладает очень ограниченной кастомизируемостью, и основными его достоинствоми ялвяются простота в использовании и относительная устойчиовость к дефектам изображения. Средняя скорость и точность делают инструмент универсальным, но не заточенным под конкретную задачу. Требуется вспомогательный код по разметке страницы и/или сортировке фрагментов текста.

Surya OCR - медленный пакет. Хотя он показывает неплохую точность распознавания непосредственно слов, так же как и EasyOCR он выдает неупорядоченный текст. Готовый набор команд, в том числе, по разметке страницы может быть интересен для рассмотрения в качестве медленного валидирующего инструмента, используемого редко, при условии, что удастся подтвердить его точность. Кастомизируемость выше, чем у EasyOCR, но тоже требует дополнительной оценки.

1. https://www.trustradius.com/ocr-software
2. https://github.com/tesseract-ocr/tesseract
3. https://github.com/JaidedAI/EasyOCR
4. https://github.com/VikParuchuri/surya
5. https://muthu.co/all-tesseract-ocr-options/
6. https://www.kaggle.com/code/dhorvay/pytesseract-function-parameters
7. https://en.wikipedia.org/wiki/Levenshtein_distance
8. https://en.wikipedia.org/wiki/Dice-S%C3%B8rensen_coefficient
